# HW 5

In this homework, we will use Bank credit scoring dataset from here.
https://www.kaggle.com/datasets/kapturovalexander/bank-credit-scoring/data

## Question 1

Install Pipenv
What's the version of pipenv you installed?
Use --version to find out


In [2]:
!pipenv --version

pipenv, version 2023.10.24


## Question 2
Use Pipenv to install Scikit-Learn version 1.3.1
What's the first hash for scikit-learn you get in Pipfile.lock?

Answer - 0402638c9a7c219ee52c94cbebc8fcb5eb9fe9c773717965c1f4185588ad3107



## Models
We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

features = ['job','duration', 'poutcome']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression().fit(X, y)
Note: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download them:

DictVectorizer
LogisticRegression
With wget:

PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework
wget $PREFIX/model1.bin
wget $PREFIX/dv.bin

## Question 3
Let's use these models!

Write a script for loading these models with pickle
Score this client:
{"job": "retired", "duration": 445, "poutcome": "success"}
What's the probability that this client will get a credit?

0.162
0.392
0.652
0.902
If you're getting errors when unpickling the files, check their checksum:

$ md5sum model1.bin dv.bin
8ebfdf20010cfc7f545c43e3b52fc8a1  model1.bin
924b496a89148b422c74a62dbc92a4fb  dv.bin

In [2]:
import pickle

In [8]:
def load(filename):
    with open(filename, 'rb') as f_in:
        return pickle.load(f_in)

dv = load('dv.bin')
model = load('model1.bin')

customer = {"job": "retired", "duration": 445, "poutcome": "success"}
X_test = dv.transform([customer])

y_pred = model.predict_proba(X_test)[0,1]
y_pred 

/Users/yash/miniconda3/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.3.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/yash/miniconda3/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


0.9019309332297606

# Question 4
Now let's serve this model as a web service

Install Flask and gunicorn (or waitress, if you're on Windows)
Write Flask code for serving the model
Now score this client using requests:
url = "YOUR_URL"
client = {"job": "unknown", "duration": 270, "poutcome": "failure"}
requests.post(url, json=client).json()
What's the probability that this client will get a credit?

0.140
0.440
0.645
0.845
Docker
Install Docker. We will use it for the next two questions.

For these questions, we prepared a base image: svizor/zoomcamp-model:3.10.12-slim. You'll need to use it (see Question 5 for an example).

This image is based on python:3.10.12-slim and has a logistic regression model (a different one) as well a dictionary vectorizer inside.

This is how the Dockerfile for this image looks like:

FROM python:3.10.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
We already built it and then pushed it to svizor/zoomcamp-model:3.10.12-slim.

Note: You don't need to build this docker image, it's just for your reference.